In [ ]:
pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
mkdir ~/.kaggle

In [ ]:
cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                             title                                       size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                             9KB  2022-10-06 08:55:25          13777        421  1.0              
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset      11KB  2022-09-30 04:05:11          10703        334  1.0              
thedevastator/240000-household-electricity-consumption-records  Household Electricity Consumption            3MB  2022-10-24 01:22:40            976         30  1.0              
akshaydattatraykhare/data-for-admission-in-the-university       Data for Admission in the University     

In [ ]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.66G/9.66G [04:45<00:00, 38.7MB/s]
100% 9.66G/9.66G [04:45<00:00, 36.3MB/s]


In [ ]:
!unzip diabetic-retinopathy-level-detection.zip

Archive:  diabetic-retinopathy-level-detection.zip
  inflating: inception-diabetic.h5   
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfb17a7cc8d4.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfdbaef73a8b.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfed7c1172ec.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cff262ed8f4c.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cffc50047828.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d02b79fc3200.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d0926ed2c8e5.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d160ebef4117.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e39b9d6f0.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e59a2b33a.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d18f64

In [ ]:
imageSize = [299,299]
trainPath = r"/content/preprocessed dataset/preprocessed dataset/training"
testPath = r"/content/preprocessed dataset/preprocessed dataset/testing"

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.xception import Xception,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True,)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                                target_size = (299,299),
                                                batch_size = 32,
                                                class_mode = 'categorical')



Found 3662 images belonging to 5 classes.


In [ ]:
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 734 images belonging to 5 classes.


In [ ]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 3s 0us/step


In [ ]:
# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data= test_set,
  epochs=30,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
3/3 [==============================] - 60s 18s/step - loss: 10.8778 - accuracy: 0.2188
Epoch 2/30
3/3 [==============================] - 50s 15s/step - loss: 13.8249 - accuracy: 0.5729
Epoch 3/30
3/3 [==============================] - 51s 15s/step - loss: 8.3021 - accuracy: 0.6250
Epoch 4/30
3/3 [==============================] - 49s 15s/step - loss: 8.8645 - accuracy: 0.4583
Epoch 5/30
3/3 [==============================] - 49s 15s/step - loss: 6.6116 - accuracy: 0.5208
Epoch 6/30
3/3 [==============================] - 49s 14s/step - loss: 5.9837 - accuracy: 0.5521
Epoch 7/30
3/3 [==============================] - 49s 15s/step - loss: 4.3489 - accuracy: 0.6146
Epoch 8/30
3/3 [==============================] - 51s 16s/step - loss: 3.1509 - accuracy: 0.6562
Epoch 9/30
3/3 [==============================] - 49s 14s/step - loss: 3.4932 - accuracy: 0.6562
Epoch 10/30
3/3 [==============================] - 50s 15s/step - loss: 4.3183 - accuracy: 0.6354
Epoch 11/30
3/3 [==========

In [ ]:
model.save('Updated-Xception-diabetic-retinopathy.h5')